In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
#!pip install keras-self-attention
#!pip install attention
#!pip install keras-multi-head
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df  = pd.read_csv("../input/tabular-playground-series-mar-2022/train.csv")


In [ ]:
print(len(df.time.unique()))
print(df.x.unique())
print(df.y.unique())
print(df.direction.unique())

In [ ]:
from sklearn.preprocessing import MinMaxScaler
#scaling data and formating them such as we only keep position (x,y), direction, hour and day of the week as features for congestion 
data = df.pivot_table(index = 'time', columns = ['direction','x','y'], fill_value = 0, values = 'congestion')

print(data)


datatime = pd.to_datetime(data.index)

sc = MinMaxScaler()
datanorm = sc.fit_transform(data)

Time = (datatime.hour.to_numpy()*3 + datatime.minute.to_numpy()/20)

datanorm= np.append(datanorm,datatime.weekday.to_numpy()[:,np.newaxis]/6, axis = 1) 
datanorm= np.append(datanorm, Time[:,np.newaxis]/71, axis = 1) 

print(datanorm)




In [ ]:
import matplotlib.pyplot as plt
#computing mean congestion per day of week and standard deviation associated

datatarget = datanorm[(datanorm[:,-2] == 0) & (datanorm[:,-1] <= 1) & (0.5 < datanorm[:,-1]) ] 

datatarget = np.reshape(datatarget[:,:-2], (26,36,65))*100
datamean = np.mean(datatarget, axis = 0)
datadev = np.empty((36,65))
for i in range(65):
    
    for j in range(36):
        A = (np.sum((datatarget[:,j,i] - datamean[j,i]), axis = 0)/26)
        datadev[j,i] = A*10**15
print(datadev)
print(np.mean(datadev, axis = 1))

In [ ]:
#formating datas as time-serie: 2 days of history for 12 hours of prediction

Ndays = 2
lenseq = int(3*24*Ndays-36)
Xtrain = np.empty((1,lenseq, datanorm.shape[1]))
Ytrain = np.empty((1,3*12, datanorm.shape[1]-2))

T = 0
for t in range(0, datanorm.shape[0]-(lenseq + 12*12), 6) :
    if t//1000 > T:
        T = t//1000
        print(T)
    Xtemp = np.reshape(datanorm[t:t+lenseq,:],(1,lenseq, datanorm.shape[1]))
    Ytemp = np.reshape(datanorm[t+lenseq:t+lenseq+3*12,:datanorm.shape[1]-2],(1,3*12, datanorm.shape[1]-2))
        
    Xtrain = np.append(Xtrain,Xtemp, axis = 0)
    Ytrain = np.append(Ytrain,Ytemp, axis = 0)
    #Ytrain.append(Ytemp)

Xtrain = Xtrain[1:,:,:]   
Ytrain = Ytrain[1:,:,:]  

Xtest = np.reshape(datanorm[-lenseq:,:],(1,lenseq, datanorm.shape[1]))

Xval = np.reshape(datanorm[-lenseq-6*12:-6*12,:],(1,lenseq, datanorm.shape[1]))

Yval = np.reshape(datanorm[-12*6:-12*3,:datanorm.shape[1]-2],(1,3*12, datanorm.shape[1]-2))
print(Xval.shape)
print(Yval.shape)


In [ ]:
print(Xtrain.shape)
print(Ytrain.shape)
print(np.isnan(Xtrain).sum())
Xtrain[np.isnan(Xtrain)] = 0
print(np.isnan(Xtrain).sum())


In [ ]:
import tensorflow as tf
tf.keras.backend.clear_session()
filt = 32
units = 128
#building and training model (bi-lstm + MultiHead Self Attention cells x 3 )

Inputs = tf.keras.Input(shape=(lenseq, datanorm.shape[1]))
x = tf.keras.layers.Reshape((lenseq,datanorm.shape[1]),  input_shape=(lenseq, datanorm.shape[1]))(Inputs)
y = tf.keras.layers.Bidirectional(tf.compat.v1.keras.layers.CuDNNLSTM(units*2, return_sequences = True))(x)
y = tf.keras.layers.Bidirectional(tf.compat.v1.keras.layers.CuDNNLSTM(units, return_sequences = True))(y)

z = tf.keras.layers.MultiHeadAttention(num_heads=9, key_dim=12, attention_axes=(1,2))(y, y, return_attention_scores=False)
z = tf.keras.layers.LayerNormalization()(z)
z2 = tf.keras.layers.Add()([y, z])
z3= tf.keras.layers.TimeDistributed(
        tf.keras.layers.Dense(units*2, activation = 'relu'))(z2)
a = tf.keras.layers.MultiHeadAttention(num_heads=9, key_dim=12, attention_axes=(1,2))(z3, z3, return_attention_scores=False)
a = tf.keras.layers.LayerNormalization()(a)
a2 = tf.keras.layers.Add()([a, z2])
a3= tf.keras.layers.TimeDistributed(
        tf.keras.layers.Dense(units*2, activation = 'relu'))(a2)
b = tf.keras.layers.MultiHeadAttention(num_heads=9, key_dim=12, attention_axes=(1))(a3, a3, return_attention_scores=False)
b = tf.keras.layers.LayerNormalization()(b)
b2 = tf.keras.layers.Add()([a2, b])
b3= tf.keras.layers.TimeDistributed(
        tf.keras.layers.Dense(units*2, activation = 'relu'))(b2)
c2= tf.compat.v1.keras.layers.CuDNNLSTM(65,return_sequences = True )(b3)
out = tf.keras.layers.Lambda(lambda x: x[:, -3*12:, :])(c2)

model = tf.keras.Model(inputs=Inputs, outputs=out)

optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.MeanAbsoluteError()
model.compile(optimizer = optimizer, loss = loss, metrics = ['mae'])
model.summary()
model.fit(Xtrain, Ytrain, shuffle = 'True', validation_data = (Xval,Yval),epochs =130, batch_size = 32)

In [ ]:
import matplotlib.pyplot as plt
fig, ax = plt.subplots( layout='constrained')
plt.ylim([0, 0.16])
ax.plot((model.history.history['loss']), label='train')

ax.plot((model.history.history['val_loss']), label='val')


In [ ]:
#import matplotlib as mpl
import matplotlib.pyplot as plt
#making predictions on validation set and comparing them too real values


Ypredval = model.predict(Xval)
Ypredval = sc.inverse_transform(Ypredval[0,:,:])
Yval2 = sc.inverse_transform(Yval[0,:,:])
TOT = 0
for i in range(0,65):
    tot = abs(Ypredval[:,i] - Yval2[:,i]).sum()/Yval2.shape[0]
    TOT += tot
    fig, ax = plt.subplots( layout='constrained')
    plt.ylim([0, 100])
    ax.plot((Ypredval[:,i]), label='pred')
    ax.plot(Yval2[:,i], label='real') 
    print(tot)
print("total =" + str(TOT/65))

In [ ]:
#import matplotlib as mpl
#ploting predictions for test
import matplotlib.pyplot as plt

Xtest = np.reshape(datanorm[-lenseq:,:],(1,lenseq, datanorm.shape[1]))
Ypred = model.predict(Xtest)
print(Ypred.shape)
Ypred = sc.inverse_transform(Ypred[0,:,:])
for i in range(0,65):
    fig, ax = plt.subplots( layout='constrained')
    plt.ylim([0, 100])
    ax.plot((Ypred[:,i]), label='pred')
    ax.plot(datamean[:,i], label='mean')  

In [ ]:
#Writing predictions in final pandas dataframe


import pandas as pd
import numpy as np
dftest  = pd.read_csv("../input/tabular-playground-series-mar-2022/test.csv")

dftest2 = dftest.sort_values(['time','direction','x','y'])
print(dftest2)
print(dftest2.shape)
Ypred2 = np.ravel(Ypred[:36,:])
print(Ypred2)
print(len(Ypred2))
dftest2.insert(1, 'congestion', Ypred2)
dftest2 = dftest2.drop(columns = ['direction','x','y','time'])
print(dftest2)
submission = dftest2.sort_values(['row_id'])



In [ ]:
#print(submission)
submission['congestion'] = round(submission['congestion'] )
print(submission)

In [ ]:
# = pd.DataFrame(Ypred)
submission.to_csv("./submission.csv", index = False)